In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/idd-dataset/IDD_Detection_Organized/val/annotations/0006735.xml
/kaggle/input/idd-dataset/IDD_Detection_Organized/val/annotations/0013425.xml
/kaggle/input/idd-dataset/IDD_Detection_Organized/val/annotations/000217_r.xml
/kaggle/input/idd-dataset/IDD_Detection_Organized/val/annotations/0001230.xml
/kaggle/input/idd-dataset/IDD_Detection_Organized/val/annotations/000348_r.xml
/kaggle/input/idd-dataset/IDD_Detection_Organized/val/annotations/001608_r.xml
/kaggle/input/idd-dataset/IDD_Detection_Organized/val/annotations/0002760.xml
/kaggle/input/idd-dataset/IDD_Detection_Organized/val/annotations/0005415.xml
/kaggle/input/idd-dataset/IDD_Detection_Organized/val/annotations/000888_r.xml
/kaggle/input/idd-dataset/IDD_Detection_Organized/val/annotations/0003840.xml
/kaggle/input/idd-dataset/IDD_Detection_Organized/val/annotations/0004155.xml
/kaggle/input/idd-dataset/IDD_Detection_Organized/val/annotations/0004620.xml
/kaggle/input/idd-dataset/IDD_Detection_Organized/val/annota

#### Installing dependencies

In [2]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 871.6/871.6 kB 19.3 MB/s eta 0:00:0000:01


In [3]:
!pip install -U ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.4/214.4 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 54.4 MB/s eta 0:00:00:00:01
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.8
    Uninstalling widgetsnbextension-3.6.8:
      Successfully uninstalled widgetsnbextension-3.6.8
  Attempting uninstall: jupyterlab-widgets
    Found existing installation: jupyterlab_widgets 3.0.11
    Uninstalling jupyterlab_widgets-3.0.11:
      Successfully uninstalled jupyterlab_widgets-3.0.11
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 

In [4]:
!pip install wandb

#### Importing libraries

In [7]:
import os
import xml.etree.ElementTree as ET
import cv2
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from ultralytics import YOLO
import wandb
import shutil

In [ ]:
# Set up paths
base_dataset_path = '/kaggle/input/idd-dataset/IDD_Detection_Organized'
output_base_path = '/kaggle/working/IDD_Detection_YOLO'
subsets = ['train', 'val']
data_yaml_path = '/kaggle/working/data.yaml'

# Define classes and ignored classes
classes = {
    'person': 0,
    'rider': 1,
    'motorcycle': 2,
    'bicycle': 3,
    'autorickshaw': 4,
    'car': 5,
    'truck': 6,
    'bus': 7,
    'traffic light': 8,
    'traffic sign': 9
}

classes_to_ignore = ['traffic sign', 'bicycle', 'traffic light']
indices_to_ignore = {classes[class_name] for class_name in classes_to_ignore}

#### Utils

In [8]:
# Function to convert XML annotations to YOLO format
def convert_annotation_to_yolo_format(xml_file, txt_file, img_width, img_height):
    """Convert XML annotations to YOLO format and save them."""
    tree = ET.parse(xml_file)
    root = tree.getroot()

    with open(txt_file, 'w') as f:
        for obj in root.findall('object'):
            cls_name = obj.find('name').text
            if cls_name not in classes:
                continue
            cls_id = classes[cls_name]

            xmlbox = obj.find('bndbox')
            xmin = int(xmlbox.find('xmin').text)
            ymin = int(xmlbox.find('ymin').text)
            xmax = int(xmlbox.find('xmax').text)
            ymax = int(xmlbox.find('ymax').text)

            x_center = ((xmin + xmax) / 2.0) / img_width
            y_center = ((ymin + ymax) / 2.0) / img_height
            width = (xmax - xmin) / float(img_width)
            height = (ymax - ymin) / float(img_height)

            f.write(f"{cls_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")

def process_annotations_and_images(subset):
    """Process annotations and images for a given subset (train/val)."""
    images_path = os.path.join(base_dataset_path, subset, 'images')
    annotations_path = os.path.join(base_dataset_path, subset, 'annotations')
    yolo_labels_path = os.path.join(output_base_path, subset, 'labels')
    yolo_images_path = os.path.join(output_base_path, subset, 'images')

    os.makedirs(yolo_labels_path, exist_ok=True)
    os.makedirs(yolo_images_path, exist_ok=True)

    for annotation_file in os.listdir(annotations_path):
        if annotation_file.endswith('.xml'):
            xml_file = os.path.join(annotations_path, annotation_file)
            image_file = os.path.join(images_path, annotation_file.replace('.xml', '.jpg'))
            txt_file = os.path.join(yolo_labels_path, annotation_file.replace('.xml', '.txt'))

            if not os.path.exists(image_file):
                print(f"Image file not found for annotation: {annotation_file}")
                continue
            
            # Copy image to YOLO folder
            shutil.copy2(image_file, yolo_images_path)

            # Load image to get dimensions
            img = cv2.imread(image_file)
            img_height, img_width, _ = img.shape

            convert_annotation_to_yolo_format(xml_file, txt_file, img_width, img_height)

    print(f"All annotations and images for {subset} have been processed successfully!")

def remove_empty_labels(labels_path):
    """Remove empty or corrupted label files."""
    empty_labels = [f for f in os.listdir(labels_path) if os.path.getsize(os.path.join(labels_path, f)) == 0]
    for label_file in empty_labels:
        os.remove(os.path.join(labels_path, label_file))
        print(f"Removed corrupted label file: {label_file}")

def update_labels(labels_path):
    """Update label files to exclude specified classes."""
    for label_file in os.listdir(labels_path):
        if label_file.endswith('.txt'):
            file_path = os.path.join(labels_path, label_file)
            with open(file_path, 'r') as f:
                lines = f.readlines()
            
            updated_lines = [line for line in lines if int(line.split()[0]) not in indices_to_ignore]
            
            with open(file_path, 'w') as f:
                f.writelines(updated_lines)

def train_model_with_attention(model, model_name):
    """Train a YOLO model with attention mechanisms."""
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    results = model.train(
        data=data_yaml_path,
        epochs=100,  
        batch=16,  
        imgsz=640,  
        lr0=0.001,  
        optimizer='Adam',  
        momentum=0.937,
        weight_decay=0.0005,
        name=model_name,
        device=device,
        project='IDD'
    )

    # Save the trained model
    model.save(f'/kaggle/working/{model_name}.pt')

    # Validate the model
    val_results = model.val()
    print(f"Validation mAP50: {val_results.box.map50:.4f}, Precision: {val_results.box.mp:.4f}, Recall: {val_results.box.mr:.4f}")

# Process data for both training and validation
for subset in subsets:
    process_annotations_and_images(subset)
    remove_empty_labels(os.path.join(output_base_path, subset, 'labels'))
    update_labels(os.path.join(output_base_path, subset, 'labels'))


# Define SE and CBAM blocks
class SEBlock(nn.Module):
    def __init__(self, channel, reduction=16):
        super(SEBlock, self).__init__()
        self.fc1 = nn.Linear(channel, channel // reduction, bias=False)
        self.fc2 = nn.Linear(channel // reduction, channel, bias=False)
        self.relu = nn.ReLU(inplace=True)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        b, c, _, _ = x.size()
        y = x.view(b, c, -1).mean(dim=2)
        y = self.fc1(y)
        y = self.relu(y)
        y = self.fc2(y)
        y = self.sigmoid(y).view(b, c, 1, 1)
        return x * y.expand_as(x)

class YOLOv8nSE(YOLO):
    def __init__(self, se_reduction=16):
        super(YOLOv8nSE, self).__init__("yolov8n.pt")
        self.se_reduction = se_reduction
        self.add_se_blocks()

    def add_se_blocks(self):
        for name, module in self.model.named_modules():
            if isinstance(module, nn.Conv2d) and module.in_channels == module.out_channels:
                se_block = SEBlock(module.out_channels, reduction=self.se_reduction)
                setattr(self.model, name, nn.Sequential(module, se_block))

class ChannelAttention(nn.Module):
    def __init__(self, in_planes, ratio=16):
        super(ChannelAttention, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
        self.fc1 = nn.Conv2d(in_planes, in_planes // ratio, kernel_size=1, bias=False)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Conv2d(in_planes // ratio, in_planes, kernel_size=1, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = self.fc2(self.relu1(self.fc1(self.avg_pool(x))))
        max_out = self.fc2(self.relu1(self.fc1(self.max_pool(x))))
        out = avg_out + max_out
        return self.sigmoid(out)

class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()
        assert kernel_size in (3, 7), 'kernel size must be 3 or 7'
        padding = 3 if kernel_size == 7 else 1
        self.conv1 = nn.Conv2d(2, 1, kernel_size, padding=padding, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        x = torch.cat([avg_out, max_out], dim=1)
        x = self.conv1(x)
        return self.sigmoid(x)

class CBAMBlock(nn.Module):
    def __init__(self, channel, reduction=16, kernel_size=7):
        super(CBAMBlock, self).__init__()
        self.ca = ChannelAttention(channel, reduction)
        self.sa = SpatialAttention(kernel_size)

    def forward(self, x):
        x = self.ca(x) * x
        x = self.sa(x) * x
        return x

class YOLOv8nCBAM(YOLO):
    def __init__(self):
        super(YOLOv8nCBAM, self).__init__("yolov8n.pt")
        self.add_cbam_blocks()

    def add_cbam_blocks(self):
        for name, module in self.model.named_modules():
            if isinstance(module, nn.Conv2d) and module.in_channels == module.out_channels:
                cbam_block = CBAMBlock(module.out_channels)
                setattr(self.model, name, nn.Sequential(module, cbam_block))


All annotations and images for train have been processed successfully!
Removed corrupted label file: 0000900.txt
Removed corrupted label file: 000120_r.txt
Removed corrupted label file: 000264_r.txt
Removed corrupted label file: 000336_r.txt
Removed corrupted label file: 0001200.txt
Removed corrupted label file: 0001080.txt
Removed corrupted label file: 000126_r.txt
Removed corrupted label file: 001464_r.txt
Removed corrupted label file: 001098_r.txt
Removed corrupted label file: 000156_r.txt
Removed corrupted label file: 000252_r.txt
Removed corrupted label file: 000192_r.txt
Removed corrupted label file: 000108_r.txt
Removed corrupted label file: 000324_r.txt
Removed corrupted label file: 000270_r.txt
Removed corrupted label file: 000144_r.txt
Removed corrupted label file: 000090_r.txt
Removed corrupted label file: 000102_r.txt
All annotations and images for val have been processed successfully!
Removed corrupted label file: 0015180.txt
Removed corrupted label file: 000984_r.txt
Remo

#### yaml

In [9]:
def create_data_yaml():
    """Create the data.yaml file for YOLO training."""
    data_yaml_content = f"""
train: {os.path.join(output_base_path, 'train/images')}
val: {os.path.join(output_base_path, 'val/images')}

nc: {len(classes)}  # Number of classes
names: {list(classes.keys())}
"""
    with open(data_yaml_path, 'w') as file:
        file.write(data_yaml_content)
    print("data.yaml file created successfully!")

In [10]:
# Create data.yaml file
create_data_yaml()

# # Initialize Weights & Biases
# wandb.init(project='IDD', name='yolov8n-attention-based')

data.yaml file created successfully!


#### SE (Squeeze-and-Excitation) Block Implementation

In [11]:
wandb.init(project = 'IDD',name = 'yolov8n-SE-attentionbases')

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [12]:
# Initialize and train models
model_se = YOLOv8nSE()
train_model_with_attention(model_se, 'yolov8n_se_attentionbased')

100%|██████████| 6.25M/6.25M [00:00<00:00, 100MB/s]


Using device: cuda
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/kaggle/working/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cuda, workers=8, project=IDD, name=yolov8n_se_attentionbased, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=None, format=torchscript, 

100%|██████████| 755k/755k [00:00<00:00, 17.1MB/s]


Overriding model.yaml nc=80 with nc=10

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytic

/opt/conda/lib/python3.10/site-packages/ultralytics/engine/trainer.py:268: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(enabled=self.amp)
train: Scanning /kaggle/working/IDD_Detection_YOLO/train/labels... 1051 images, 32 backgrounds, 0 corrupt: 100%|██████████| 1069/1069 [00:00<00:00, 1247.46it/s]

...

lr/pg0,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr/pg1,▃███▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁
lr/pg2,▃███▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁
metrics/mAP50(B),▁▅▆▆▆▇▇▇▇▇▇▇▇▇▇▇█▇▇█▇███████████████████
metrics/mAP50-95(B),▁▅▆▆▆▇▇▆▇▇▇▇▇▇▇▇▇▇▇█▇▇▇███▇█████████████
metrics/precision(B),▁▆▇▇▇▇▇▇▇█▇▇▇█▇▇█▇▇▇▇██████▇██████▇█████
metrics/recall(B),▁▄▅▅▅▇▆▆▇▇▇▆▇▆▇▇▇▆▇▇▇▇▇▇▇█▇████▇████▇▇██
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/box_loss,█▆▅▅▅▅▄▄▄▄▃▃▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁


Ultralytics YOLOv8.2.83 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
Model summary (fused): 168 layers, 3,007,598 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /kaggle/working/IDD_Detection_YOLO/val/labels.cache... 970 images, 75 backgrounds, 0 corrupt: 100%|██████████| 977/977 [00:00<?, ?it/s]
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 62/62 [00:12<00:00,  5.16it/s]


                   all        977       6455      0.746      0.481      0.548       0.37
                person        511       1385      0.775      0.422      0.499      0.262
                 rider        436       1147      0.749      0.361      0.431      0.238
            motorcycle        500       1217      0.736      0.418      0.474      0.266
          autorickshaw        166        213      0.657      0.432      0.491      0.338
                   car        524       1246      0.707      0.516      0.565      0.401
                 truck        408        678      0.704      0.534      0.586      0.448
                   bus        389        569      0.894      0.682       0.79      0.636
Speed: 0.1ms preprocess, 1.7ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to IDD/yolov8n_se_attentionbased2
Validation mAP50: 0.5480, Precision: 0.7461, Recall: 0.4808


/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


In [15]:
# Save the trained model
model_se.save('/kaggle/working/yolov8n_idd_se-1.pt')

#### Implement CBAM (Convolutional Block Attention Module)

In [16]:
wandb.init(project = 'IDD',name = 'yolov8n-CBAM-attentionbased')

wandb: Currently logged in as: rajeshai (rajeshai-university-of-limerick). Use `wandb login --relogin` to force relogin


In [17]:
model_cbam = YOLOv8nCBAM()
train_model_with_attention(model_cbam, 'yolov8n_cbam_attentionbased')

Using device: cuda
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/kaggle/working/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cuda, workers=8, project=IDD, name=yolov8n_cbam_attentionbased, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=None, format=torchscript

train: Scanning /kaggle/working/IDD_Detection_YOLO/train/labels.cache... 1051 images, 32 backgrounds, 0 corrupt: 100%|██████████| 1069/1069 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /kaggle/working/IDD_Detection_YOLO/val/labels.cache... 970 images, 75 backgrounds, 0 corrupt: 100%|██████████| 977/977 [00:00<?, ?it/s]


Plotting labels to IDD/yolov8n_cbam_attentionbased/labels.jpg... 
optimizer: Adam(lr=0.001, momentum=0.937) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 4 dataloader workers
Logging results to IDD/yolov8n_cbam_attentionbased
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        977       6455      0.757      0.475      0.547      0.369
                person        511       1385      0.791      0.417      0.499      0.262
                 rider        436       1147      0.751      0.352      0.431      0.239
            motorcycle        500       1217      0.755      0.415      0.474      0.266
          autorickshaw        166        213      0.658      0.424      0.489      0.336
                   car        524       1246      0.728      0.514      0.563        0.4
                 truck        408        678      0.715      0.529      0.586      0.448
                   bus        389        569      0.904      0.675      0.787      0.635
Speed: 0.1ms preprocess, 1.4ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to IDD/yolov8n_cbam_attentionbased


lr/pg0,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr/pg1,▃███▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁
lr/pg2,▃███▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁
metrics/mAP50(B),▁▅▆▆▆▇▇▇▇▇▇▇▇▇▇▇█▇▇█▇███████████████████
metrics/mAP50-95(B),▁▅▆▆▆▇▇▆▇▇▇▇▇▇▇▇▇▇▇█▇▇▇███▇█████████████
metrics/precision(B),▁▆▇▇▇▇▇▇▇█▇▇▇█▇▇█▇▇▇▇██████▇██████▇█████
metrics/recall(B),▁▄▅▅▅▇▆▆▇▇▇▆▇▆▇▇▇▆▇▇▇▇▇▇▇█▇████▇████▇▇██
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/box_loss,█▆▅▅▅▅▄▄▄▄▃▃▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁


Ultralytics YOLOv8.2.83 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
Model summary (fused): 168 layers, 3,007,598 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /kaggle/working/IDD_Detection_YOLO/val/labels.cache... 970 images, 75 backgrounds, 0 corrupt: 100%|██████████| 977/977 [00:00<?, ?it/s]
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 62/62 [00:12<00:00,  5.16it/s]


                   all        977       6455      0.746      0.481      0.548       0.37
                person        511       1385      0.775      0.422      0.499      0.262
                 rider        436       1147      0.749      0.361      0.431      0.238
            motorcycle        500       1217      0.736      0.418      0.474      0.266
          autorickshaw        166        213      0.657      0.432      0.491      0.338
                   car        524       1246      0.707      0.516      0.565      0.401
                 truck        408        678      0.704      0.534      0.586      0.448
                   bus        389        569      0.894      0.682       0.79      0.636
Speed: 0.1ms preprocess, 1.5ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to IDD/yolov8n_cbam_attentionbased2
Validation mAP50: 0.5480, Precision: 0.7461, Recall: 0.4808


In [18]:
# Save the trained model
model_cbam.save('/kaggle/working/yolov8n_idd_cbam-1.pt')